In [1]:
%load_ext autotime

In [2]:
import pandas as pd
import numpy as np
import glob, shutil
import timeit, os, gc
import subprocess as sp
from tqdm import tqdm
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import json
from IPython.display import HTML
from base64 import b64encode
import cv2



time: 942 ms


In [3]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 4000)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(font_scale=1.0)

time: 499 ms


In [4]:
#um = 1
HOME = "./"
FFMPEG = "C:/Users/simon/Desktop/Code/ffmpeg/bin"
FFMPEG_PATH = FFMPEG
DATA_FOLDER = "D:/DeepFakeKaggleChallenge/Data"
#D:\CueLearning\DFData\Df4Extracted
TMP_FOLDER =  "D:/DeepFakeKaggleChallenge"
DATA_FOLDER_TRAIN = DATA_FOLDER
VIDEOS_FOLDER_TRAIN = DATA_FOLDER_TRAIN + "/dfdc_train_part_"
IMAGES_FOLDER_TRAIN = TMP_FOLDER + "/LostImages"
#AUDIOS_FOLDER_TRAIN = TMP_FOLDER + "/audios"+str(num)
#CROPPED_IMAGES_FOLDER = TMP_FOLDER + "/cropped_images"+str(num)
EXTRACT_META = True # False
EXTRACT_CONTENT = True # False
EXTRACT_FACES = True # False
FRAME_RATE = 3.0 # Frame per second to extract (max is 30.0)

time: 999 µs


In [5]:
def run_command(*popenargs, **kwargs):
    closeNULL = 0
    try:
        from subprocess import DEVNULL
        closeNULL = 0
    except ImportError:
        import os
        DEVNULL = open(os.devnull, 'wb')
        closeNULL = 1

    process = sp.Popen(stdout=sp.PIPE, stderr=DEVNULL, *popenargs, **kwargs)
    output, unused_err = process.communicate()
    retcode = process.poll()

    if closeNULL:
        DEVNULL.close()

    if retcode:
        cmd = kwargs.get("args")
        if cmd is None:
            cmd = popenargs[0]
        error = sp.CalledProcessError(retcode, cmd)
        error.output = output
        raise error
        
    return output

def ffprobe(filename, options = ["-show_error", "-show_format", "-show_streams", "-show_programs", "-show_chapters", "-show_private_data"]):
    ret = {}
    command = [FFMPEG_PATH + "/ffprobe", "-v", "error", *options, "-print_format", "json", filename]
    ret = run_command(command)
    if ret:
        ret = json.loads(ret)
    return ret

# ffmpeg -i input.mov -r 0.25 output_%04d.png
def ffextract_frames(filename, output_folder, rate = 0.25):
    command = [FFMPEG_PATH + "/ffmpeg", "-i", filename, "-r", str(rate), "-y", output_folder + "/output_%04d.png"]
    ret = run_command(command)
    return ret

# ffmpeg -i input-video.mp4 output-audio.mp3
def ffextract_audio(filename, output_path):
    command = [FFMPEG_PATH + "/ffmpeg", "-i", filename, "-vn", "-ac", "1", "-acodec", "copy", "-y", output_path]
    ret = run_command(command)
    return ret

time: 61 ms


In [6]:
#train_pd = pd.read_json(VIDEOS_FOLDER_TRAIN + "/metadata.json").T.reset_index().rename(columns={"index": "filename"})
train_pd = pd.read_pickle("lost_videos_df.pkl")
train_pd.head()

,filename,label,original,block
33671,jwerbdhrmu.mp4,REAL,NaN,21
60861,ahrqzxkppw.mp4,REAL,NaN,32
60997,bxtxxxylhv.mp4,REAL,NaN,32
61370,fvfzksrsbd.mp4,REAL,NaN,32
61805,kolwvxizei.mp4,REAL,NaN,32


time: 75 ms


In [ ]:
#AUDIO_FORMAT = "wav" # "wav"
videos_folder = VIDEOS_FOLDER_TRAIN
images_folder_path = IMAGES_FOLDER_TRAIN
#audios_folder_path = AUDIOS_FOLDER_TRAIN
if EXTRACT_CONTENT == True:
    # 1h20min for chunk#0 (11GB)
    # Extract some images + audio track
    for idx, row in tqdm(train_pd.iterrows(), total=train_pd.shape[0]):
        try:
            video_path = videos_folder + str(row["block"]) + "/" + row["filename"]
            images_path = images_folder_path + "/" + row["filename"][:-4]
            #audio_path = audios_folder_path + "/" + row["filename"][:-4]
            # Extract images
            if not os.path.exists(images_path): os.makedirs(images_path)
            ret = ffextract_frames(video_path, images_path, rate = FRAME_RATE)
            # Extract audio
            #if not os.path.exists(audio_path): os.makedirs(audio_path)
            #ret = ffextract_audio(video_path, audio_path + "/audio." + AUDIO_FORMAT)
        except:
            print("Cannot extract frames for:" + row["filename"])

 44%|████████████████████████████████▋                                         | 1814/4107 [1:35:09<1:56:26,  3.05s/it]